In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# 0. Veri Setinin Hazırlanması

In [ ]:
# Kullanacağımız kütüphaneleri yükleyelim.

# Pandas ve NumPy
import pandas as pd
import numpy as np

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Veri setini notebook içerisine dahil edelim.

df_ = pd.read_csv(filepath_or_buffer = "/kaggle/input/imdb-movies-data/imdb-movies-dataset.csv")
df = df_.copy()

In [ ]:
import pandas as pd
import numpy as np
import random

def add_random_missing_values(dataframe: pd.DataFrame,
                              missing_rate: float = 0.05) -> pd.DataFrame:
    """Turns random values to NaN in a DataFrame.

    To use this function, you need to import pandas, numpy and random libraries.

    Args:
        dataframe (pd.DataFrame): DataFrame to be processed.
        missing_rate (float): Percentage of missing value rate in float format. Defaults 0.05

    Returns:
        df_missing (pd.DataFrame): Processed DataFrame object.

    """
    # Get copy of dataframe
    df_missing = dataframe.copy()

    # Obtain size of dataframe and number total number of missing values
    df_size = dataframe.size
    num_missing = int(df_size * missing_rate)

    # Get random row and column indexes to turn them NaN
    for _ in range(num_missing):
        row_idx = random.randint(0, dataframe.shape[0] - 1)
        col_idx = random.randint(0, dataframe.shape[1] - 1)

        df_missing.iat[row_idx, col_idx] = np.nan

    return df_missing

Veri setimiz için gereken kütüphaneleri yüklüyüroz. Eksi veri setini oluşturuyoruz.                    

# 1. Veriye İlk Bakış

In [ ]:
# Veri setinin ilk 3 gözlemini görelim.
df.head(n = 3)

In [ ]:
# Sondan 5 gözlem için
df.tail(5)

In [ ]:
df.shape 

Veri setimizde 10000 satır v 15 sütün bulunmaktadır.

In [ ]:

print(list(df.columns), '\n')
print(f"Veri seti içerisinde toplam {len(df.columns)} kolon vardır.\n")

print(f"df.columns'un döndüğü veri tipi: {type(df.columns)}\n")

Veri setinde değişkenlerin isimlerini sıralarız.

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
categorical_features = []
numerical_features = []

for col in df.columns:
    if (df[col].dtype == "object") or (df[col].dtype == "categorical"):
        categorical_features.append(col)
    else:
        numerical_features.append(col)

In [ ]:
categorical_features, numerical_features

In [ ]:
for col in df.columns:
    if col in categorical_features:
        df[col] = pd.Categorical(df[col])

In [ ]:
df.info()

In [ ]:
df['Votes'][:3]

In [ ]:
df.describe().T # -> .T eklentisi elde edeceğimiz çıktıyı transpose eder.

In [ ]:
df.isna().sum() 

Veri setindeki eksik verileri söyler bize.

In [ ]:
df.isna().sum().sum()

Eksik verilerin toplamıdır.

In [ ]:
# Eksik olmayan değerlerin sayısı
df.notnull().sum()

In [ ]:

print(f"Veri seti içerisinde toplam {df.notnull().sum().sum()} adet eksik olmayan, {df.isnull().sum().sum()} eksik gözlem var.")

In [ ]:
# Veri setinde en az bir gözlemi eksik olan kayıtları gösterir.
df[df.isnull().any(axis = 1)]

In [ ]:
# Hiç eksik gözlemi bulunmayan kayıtları gösterir. 
df[df.notnull().all(axis = 1)][:5]

In [ ]:
import missingno as msno

msno.bar(df = df,
         figsize = (8, 4),
         fontsize = 10);

Eksik değerleri de gösteren bir veri analizi grafiğidir.

In [ ]:
import missingno as msno

msno.matrix(df = df[['Votes', 'Year', 'Rating']],
            figsize = (10, 6),
            fontsize = 10)

In [ ]:

msno.heatmap(df = df,
             figsize = (8, 3),
             fontsize = 8);

Değerlerler 1'e ne kadar biribirleri ile olan ilişkşileri o kadar yüksektir. 0 ise birbirini etkileyen bir durum bulunmamaktadır.

# 2. Eksik Verilerin Silinmesi

In [ ]:
import pandas as pd

# IMDb verisini yükle
df = pd.read_csv('/kaggle/input/imdb-movies-data/imdb-movies-dataset.csv')

# Her sütundaki eksik veri oranını hesapla (yüzde)
missing_data = df.isna().mean() * 100

# Her sütundaki eksik veri sayısını hesapla
missing_data_count = df.isna().sum()

# Eksik verileri yazdır
print("Eksik Veri Oranı (Yüzde olarak):")
print(missing_data)

print("\nEksik Veri Sayısı:")
print(missing_data_count)

In [ ]:
import missingno as msno

# Eksik verileri görselleştir
msno.matrix(df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Eksik ve normal verilerin oranını hesaplamak
missing_data = df.isnull().sum()  # Her sütundaki eksik veri sayısını bul
total_data = df.shape[0]  # Toplam satır sayısı
missing_percentage = (missing_data / total_data) * 100  # Eksik verilerin yüzdesi
non_missing_percentage = 100 - missing_percentage  # Tam verilerin yüzdesi

# Grafik için veri hazırlama
missing_data_summary = pd.DataFrame({
    'Missing Data': missing_percentage,
    'Non-Missing Data': non_missing_percentage
})

# Görselleştirme
missing_data_summary.plot(kind='bar', stacked=True, figsize=(12, 6), color=['red', 'green'])

# Grafik düzenlemeleri
plt.title('Eksik ve Normal Verilerin Oranı', fontsize=16)
plt.xlabel('Sütunlar', fontsize=12)
plt.ylabel('Yüzde (%)', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Veri Durumu', labels=['Eksik Veri', 'Tam Veri'])
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd

# IMDb verisini yükle
df = pd.read_csv('/kaggle/input/imdb-movies-data/imdb-movies-dataset.csv')

# Toplam eksik veri sayısını hesapla
total_missing = df.isna().sum().sum()

# Veri setindeki toplam hücre sayısını hesapla
total_cells = df.size

# Eksik veri oranını hesapla
missing_percentage = (total_missing / total_cells) * 100

# Sonuçları yazdır
print(f"Veri setindeki toplam eksik veri oranı: {missing_percentage:.2f}%")


Eksik veri oranı 5 den küçükse çok düşük veri oranı olarak kabul edilir. Silme yöntemi tercih ediir bu durumlarda.

In [ ]:
df = df.drop(['Poster', 'Description', 'Metascore', 'Review Title', 'Review'], axis=1)
df

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
print(df.dtypes)

In [ ]:
df["Year"] = df["Year"].astype(int)
df["Review Count"] = df["Review Count"].str.replace(',', '')
df["Review Count"] = df["Review Count"].astype(int)
df["Votes"] = df["Votes"].str.replace(',', '')
df["Votes"] = df["Votes"].astype(int)
df.head()


In [ ]:
df.describe().T

Eksik veriler silindikten sonra oluşan istatistiksel hesaplamadır.

# 3. Kategorik Değişken Analizi


In [ ]:
df = df_.copy()
df.info()

In [ ]:
df['Certificate'].value_counts().plot.barh();

Vizyona giren filmlerin türlerine göre dağılımını gösteriyor.

In [ ]:
years = df["Year"].unique()
years.sort()

colors = sns.color_palette("Purples", len(years))

color_mapping = {year: colors[i] for i, year in enumerate(years)}

bar_colors = df["Year"].map(color_mapping)

Freq = df["Year"].value_counts().sort_index().plot(
    kind="bar", 
    title="Distribution of Movies Released per Year", 
    figsize=(16, 8), 
    color=bar_colors
)

Freq.set_xlabel("Year")
Freq.set_ylabel("Frequency")
plt.grid(True, axis='y')
plt.show()

Vizyona giren filmlerin yıllara göre dağılımıdır. En çok film 2023 yılına aittir

In [ ]:
Distribution_Rating = df['Rating'].value_counts().sort_index()

plt.figure(figsize=(16, 8))
sns.barplot(x=Distribution_Rating.index, y=Distribution_Rating.values, palette='viridis')
plt.title("Distribution of Rating")
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.grid(True, axis='y')
plt.show()


In [ ]:
sns.set(style="darkgrid")

fig, axes = plt.subplots(figsize=(12, 10))
grouped = df.groupby("Title")
mean = pd.DataFrame(grouped["Rating"].mean())
mean1 = mean.sort_values("Rating", ascending=False).head(20)

palette = sns.color_palette("magma", len(mean1))

sns.barplot(x=mean1["Rating"], y=mean1.index, ax=axes, palette=palette)

for container in axes.containers:
    axes.bar_label(container, label_type="edge", padding=3, fontsize=10, color='black', weight='bold')


axes.set_xlabel('Average Rating', fontsize=14)
axes.set_ylabel('Movie Title', fontsize=14)
axes.set_title('Top 20 Movies by Average Rating', fontsize=16, weight='bold')

axes.grid(True, axis='x', linestyle='--', alpha=0.7)

sns.despine()

plt.tight_layout()
plt.show()


Vizyona giren filmlerin arasında en çok puana sahip 20 filmin verileridir.

In [ ]:
df.head()

In [ ]:
df_usd = df.groupby(by = "Title")["Rating"].mean().to_frame().reset_index().sort_values(by = "Rating", ascending = False)

df_usd

In [ ]:
df.groupby(by = ['Title', 'Genre'])['Rating'].mean().to_frame().reset_index().sort_values(by = "Rating", ascending = False)[:5]

* > 4.1 Sürekli Değişkenlerin İncelenmesi¶

In [ ]:

df_numerical = df.select_dtypes(include = ["float64", "int64"])
df_numerical.head()

In [ ]:
df_numerical.describe().T

In [ ]:
# Bir veri setindeki numerik kolonların istatistiksel bilgilerine erişmek için kullanılır.

def give_stats(dataframe: pd.DataFrame) -> None:
    """Prints statistical information for numerical columns.

    Args:
        dataframe (pd.DataFrame): DataFrame object.
    
    Return:
        None
    
    """

    num_df = dataframe.select_dtypes(include = ["float", "int"])

    for col in num_df.columns:
        print(f"**********{col}**********")
        print(f"Mean value of {col} is {num_df[col].mean():.2f}")
        print(f"Std value of {col} is {num_df[col].std():.2f}")
        print(f"Max value of {col} is {num_df[col].max()}")
        print(f"Min value of {col} is {num_df[col].min()}")
        print(f"Count value of {col} is {num_df[col].count()}")
        print(f"Median value of {col} is {num_df[col].median()}\n")


In [ ]:
give_stats(dataframe = df)

In [ ]:

# kdeplot ise ilgili numerik değişkenin yoğunluğunu gösterir.
sns.kdeplot(df['Rating'], fill = True);

In [ ]:
# Farklı numerik görselleştirme işlemleri için iris veri setini yükleyelim.
iris = sns.load_dataset(name = "iris")
iris[:3]

In [ ]:
# Scatter Plot, numerik değişkenler arasındaki saçılım ilişkisini gösterir.
sns.scatterplot(data = iris,
                x = "sepal_width",
                y = "sepal_length",
                hue = "species");

* > 5. Aykırı Değer Analizi (Outliers)

In [ ]:
df = df_.copy()

In [ ]:

plt.figure(figsize = (8, 6))
sns.boxplot(data = df,
            y = df["Rating"],
            orient = "v");

In [ ]:
df_Rating = df['Rating']
df_Rating

In [ ]:
# Quantile değerlerin belirlenmesi.
Q1 = df_Rating.quantile(0.25)
Q3 = df_Rating.quantile(0.75)

print(Q1)
print(Q3)

# IQR değerin belirlenmesi.
IQR = Q3-Q1
print(IQR)

# Alt ve üst sınırların belirlenmesi.
lower_fence = Q1 - 1.5*IQR
upper_fence = Q3 + 1.5*IQR

In [ ]:

outlier_idx = df_Rating[df_Rating > upper_fence].index

outlier_idx

In [ ]:
df_Rating[df_Rating > upper_fence]

In [ ]:

df_del = df[~(df_Rating > upper_fence)]

df_del

In [ ]:
plt.figure(figsize = (8, 6))
sns.boxplot(data = df_del,
            y = df_del["Rating"],
            orient = "v");